In [1]:
notebookName = 'audio-deepfake-detection-training'
runJobId = 'ASVspoof-2019_training'
# runJobId = 'ASVspoof-2019_small-eval-1'     # This should fail

In [ ]:
import configuration.configuration as configuration
import model_definitions.model_cnn_definition as model_cnn_definition
from preprocessors.mel_spectrogram import MelSpectrogramPreprocessor
from notebook_utils import notebookToPython
from processors.basic_model_training_processor import BasicModelTrainingProcessor
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor

In [ ]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

if (job.newModelGenerated == False):
    raise ValueError("This notebook is meant for training. Select a job without a value for 'persisted-model' set.")

In [ ]:
generator = MelSpectrogramPreprocessor()
X, y_encoded = generator.extract_features_multipleSource(job, job.dataPathSuffix)

In [ ]:
trainingProc = BasicModelTrainingProcessor(job, model_cnn_definition.ModelCnnDefinition)
model, X_train, X_test, y_train, y_test = trainingProc.process(X, y_encoded, 1)

### Test Model

In [ ]:
evaluationProc = BasicModelEvaluationProcessor(job, model)
evaluationProc.process(X_test, y_test)


In [ ]:
print("\n")
report = evaluationProc.reportSnapshot(trainingProc)
evaluationProc.writeReportToFile(job.persistedModelResults, report)